In [2]:
using Plots
using SparseArrays
using LinearAlgebra

In [3]:
function source_function(x) #To change tomorrow but it works 
   #= X0 = 20.0
    σ = 0.8
    A = 10.
    return  A .* exp.(.-((x .- X0).^2)./σ^2)  =#
    return 10. .+( x .* 0.)
end

source_function (generic function with 1 method)

In [6]:
function source(N)
#Parameter
    l = 80.
    dx = l/N
    
#Mesh
    X_vect = Vector(0:dx:l)
    #print(X_vect) 
    
# Function source
    f = source_function(X_vect)
    #plot(X_vect,f)
    return f
    
end

source (generic function with 1 method)

In [7]:
function matrix(n)
#Parameter
    l = 80.
    σ = 10 #contrainte GPa
    E = 128 #contrainte Gpa
    ϵ = σ/E #Hooke 
    dx = l/n
    α = ϵ / dx^4
    

# Build the matrix
    A = spdiagm( -1 => -4 *α* ones(n-1), -2 =>  α* ones(n-2), 0 =>  6  *α* ones(n)   , 1 => - 4 *α* ones(n-1), 2 =>   α*ones(n-2) )

# Coefficient that change in the  matrix 
    A[1,2] = 0
    A[2,1] = 0
    A[1,1]= 1
    A[1,3] = 0
    A[2,2] = 1
    A[2,3] = 0
    A[2,4] = 0
    A[n-1, n-3] = 0
    A[n-1,n-2] = 0
    A[n-1,n-1] = 1
    A[n-1,n] = 0
    A[n,n-2] = 0
    A[n,n-1] = 0
    A[n,n]= 1

#time 135.690 microseconds and 65 allocations for n= 2000
    return A
end

matrix (generic function with 1 method)

In [8]:
function exact_solution(N) #With a constant source F=10 for example
    l= 80 #length
    dx = l/ N
    σ = 10 #contrainte Gpa
    E = 128 #contrainte GPA
    ϵ = σ/E #hooke 
    u_exact = zeros(N+1)
    x = 0
    for i in 1:N+1
        u_exact[i] = (10/(24*ϵ))* x^2 * (x-l)^2 #To change tomorrow to suppress the loop for but it works 
        x = x + dx     
    end
    return u_exact
end

exact_solution (generic function with 1 method)

In [9]:
function biharmonic(N)

    #Parameter of the beam
        l= 80 #length
        dx = l/ N  

    # Build of the matrix A
        A = matrix(N+1)
        #print(A)

    # Vector B from Au = B (Boundary)
        b = zeros(N+1)
        b[1] = 0
        b[2] = 0
        b[N] = 0
        b[end] = 0
        #print(b)

    # Vector S for the source power
        X_vect = Vector(0:dx:l)
        #print(length(X_vect))
        S = source_function(X_vect) 
        #print(length(S))
        
    # Solving of the Linear System
        u = A \ (b+S) 
        #u = A \ b  
        #plot(X_vect, u)
        return u 
    end 


biharmonic (generic function with 1 method)

In [10]:
function curves(N) #with 500 points, the two curves overlap so the resoltion works 
        l = 80.0
        dx = l/ N 
        exact = exact_solution(N)
        modele = biharmonic(N)
        X = [i for i in 0:dx:l] 
        plot(X, [exact,modele])
        #print(length(exact), length(modele), length(X))
        #print(X)
    end 

curves (generic function with 1 method)